In [1]:
import os
import shutil
import stat
from PIL import Image, ImageOps
import os
import glob 
import hashlib
import numpy as np
import torch
from torchvision import transforms

In [2]:
def convert_png(directory, label, target):
    for i, image in enumerate(os.listdir(directory)):
        f = os.path.join(directory, image)
        if os.path.isfile(f):
            image = Image.open(f)
            image.save(f'data/{i}{label}.{target}')
            
    os.system('rmdir /S /Q "{}"'.format(directory))

In [ ]:
"""
    DO NOT RUN
"""

directory_1 = 'data/cancer'
directory_2 = 'data/non-cancer'
target = '.png'

convert_png(directory_1, 1, target)
convert_png(directory_2, 0, target)

In [3]:
class ImagePreprocessor:
    """
    A class for preprocessing images.

    Args:
        image_dir (str): The directory containing the images.
        target_size (tuple): The target size of the images after resizing.
        normalize_range (tuple): The range to normalize the pixel values to.

    Attributes:
        image_dir (str): The directory containing the images.
        target_size (tuple): The target size of the images after resizing.
        normalize_range (tuple): The range to normalize the pixel values to.
        image_hashes (dict): A dictionary to store the hashes of processed images.

    Methods:
        hash_image(image): Computes the hash of an image.
        check_duplicates(): Checks for duplicate images and augments them if necessary.
        augment_image(image): Augments an image by applying random transformations.
        resize_image(image): Resizes an image to the target size.
        normalize_image(image): Normalizes the pixel values of an image.
        convert_to_tensor(image): Converts an image to a PyTorch tensor.
        preprocess_image(image): Preprocesses an image by resizing, normalizing, and converting to a tensor.
    """

    def __init__(self, image_dir, target_size, normalize_range):
        """
        Initializes the SmileSavior object.

        Args:
            image_dir (str): The directory containing the images.
            target_size (tuple): The desired size of the images.
            normalize_range (tuple): The range to normalize the pixel values to.

        Attributes:
            image_dir (str): The directory containing the images.
            target_size (tuple): The desired size of the images.
            normalize_range (tuple): The range to normalize the pixel values to.
            image_hashes (dict): A dictionary to store the image hashes.

        Returns:
            None
        """
        self.image_dir = image_dir
        self.target_size = target_size
        self.normalize_range = normalize_range
        self.image_hashes = {}

    def hash_image(self, image):
        """
        Calculates the MD5 hash of the given image.

        Parameters:
        image (numpy.ndarray): The image to be hashed.

        Returns:
        str: The MD5 hash of the image.
        """
        return hashlib.md5(image.tobytes()).hexdigest()

    def check_duplicates(self):
        """
        Check for duplicate images in the specified image directory.

        This method iterates through all the images in the directory and checks if their hash values
        already exist in the `image_hashes` dictionary. If a duplicate image is found, it is augmented
        and saved back to the same location.

        Note: The `hash_image` and `augment_image` methods are assumed to be defined elsewhere.

        Returns:
            None
        """
        images = glob.glob(os.path.join(self.image_dir, '*'))
        for image_path in images:
            image = Image.open(image_path).convert('RGB')
            image_hash = self.hash_image(image)
            
            if image_hash in self.image_hashes:
                image = self.augment_image(image)
                image.save(image_path)
            else:
                self.image_hashes[image_hash] = image_path

    def augment_image(self, image):
        """
        Apply a random augmentation to the given image.

        Parameters:
        image (PIL.Image.Image): The input image to be augmented.

        Returns:
        PIL.Image.Image: The augmented image.
        """
        augmentations = [ImageOps.mirror, ImageOps.flip, ImageOps.rotate(image, 90)]
        return augmentations[np.random.randint(0,len(augmentations))](image)
    
    def resize_image(self, image):
        """
        Resizes the given image to the target size.

        Parameters:
        - image: The image to be resized.

        Returns:
        - The resized image.
        """
        return image.resize(self.target_size)

    def normalize_image(self, image):
        """
        Normalize the given image.

        Args:
            image (PIL.Image.Image): The input image to be normalized.

        Returns:
            PIL.Image.Image: The normalized image.

        """
        image_array = np.array(image).astype(np.float32) #Convert image to numpy array and float32 type. need to conver to float32 type because image data is typically in integer format and we need float precision
        image_array /= 255.0 #Because each pixel value is between 0 and 255, divide by 255 to normalize the pixel values to the range [0, 1]
        if self.normalize_range == (-1, 1): 
            image_array = image_array * 2 - 1 #basically scales the values from [0, 1] to [-1, 1]
        return Image.fromarray((image_array * 255).astype(np.uint8)) #convert back to PIL
    
    def convert_to_tensor(self, image):
        """
        Converts an image to a PyTorch tensor.

        Args:
            image (PIL.Image.Image): The input image.

        Returns:
            torch.Tensor: The converted tensor.
        """
        transform = transforms.ToTensor()
        return transform(image)
    
    def preprocess_image(self, image):
        """
        Preprocesses an image by performing the following steps:
        1. Checks for duplicates in the image directory.
        2. Retrieves a list of images in the directory.
        3. Processes each image by resizing, normalizing, and converting it to a tensor.
        4. Appends the processed image to a list.
        
        Args:
            image: The input image to be preprocessed.
            
        Returns:
            A list of processed images.
        """
        self.check_duplicates()
        images = glob.glob(os.path.join(self.image_dir, '*'))
        processed_images = []
        for image_path in images:
            image = Image.open(image_path).convert('RGB')
            image = self.resize_image(image)
            image = self.normalize_image(image)
            image = self.convert_to_tensor(image)
            processed_images.append(image)
        return processed_images

In [4]:
targetSize = (256,256)
normalizeRange = (-1, 1)
directory = '/data/'
preprocessor1 = ImagePreprocessor('directory', targetSize, normalizeRange)

processed_images = preprocessor1.preprocess_image(directory)